# Tutorial 1 - RNN Sequence classifier

In this notebook, we will predict the winner of a basketball game based on the scores observed in the first 3 quarters of the game. Each column represents the beginning of a minute during the game. (There are 12 minutes in each quarter. There are  3 quarters in the data, so we have 36 columns as input variables in chronological order.) The values captured in each column represent the score difference observed at that minute (home score minus away score)<br><br>

The last column `W` represents whether the home team (1) or the away team (0) won the game. This is the target variable. <br><br>
**Our unit of analysis is a single game.**

In [1]:
# Common imports
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd

np.random.seed(1)
tf.random.set_seed(1)

## Get the data

In [2]:
data = pd.read_csv("basketball.csv")

In [3]:
data.shape

(1230, 37)

In [4]:
data.head()

,M1,M2,M3,M4,M5,M6,M7,M8,M9,M10,...,M28,M29,M30,M31,M32,M33,M34,M35,M36,W
0,-2,-1,1,1,-1,-3,-1,0,3,6,...,9,11,10,7,7,4,6,2,1,1
1,0,2,7,6,10,8,8,6,0,6,...,7,11,11,14,14,15,13,13,13,0
2,0,-2,2,0,5,4,5,3,5,3,...,9,11,13,13,12,17,15,15,12,1
3,0,2,0,3,4,3,5,4,3,1,...,10,6,7,7,8,8,8,8,8,1
4,0,-2,-2,0,3,-2,-7,-5,-7,-4,...,10,10,15,13,11,11,11,13,10,1


In [5]:
y = data['W']
X = data.drop('W', axis=1)

## Split the data

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

## Data Transformation

In [7]:
#Target variables need to be an array with integer type
y_train = np.array(y_train)
y_test = np.array(y_test)

y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

In [8]:
#Check the first 10 values of the train_y data set
y_train[0:10]

array([0, 1, 1, 1, 1, 1, 0, 0, 0, 0])

In [9]:
#Convert input variables to a 2-D array with float data type
X_train = np.array(X_train)
X_test = np.array(X_test)

X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)

In [10]:
X_train

array([[ -3.,  -3.,  -2., ...,  -5.,  -3.,  -7.],
       [  0.,   1.,  -1., ...,  10.,  12.,  15.],
       [ -3.,  -2.,  -2., ...,  14.,  12.,  12.],
       ...,
       [  2.,   6.,   8., ...,  -2.,   1.,   0.],
       [ -2.,   1.,  -1., ..., -19., -21., -14.],
       [  2.,   1.,   3., ...,   0.,   2.,   4.]], dtype=float32)

In [11]:
#Keras expects a different input format:
#Data needs to have 3 dimensions

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [12]:
X_train.shape, y_train.shape

((861, 36, 1), (861,))

In [13]:
X_train

array([[[ -3.],
        [ -3.],
        [ -2.],
        ...,
        [ -5.],
        [ -3.],
        [ -7.]],

       [[  0.],
        [  1.],
        [ -1.],
        ...,
        [ 10.],
        [ 12.],
        [ 15.]],

       [[ -3.],
        [ -2.],
        [ -2.],
        ...,
        [ 14.],
        [ 12.],
        [ 12.]],

       ...,

       [[  2.],
        [  6.],
        [  8.],
        ...,
        [ -2.],
        [  1.],
        [  0.]],

       [[ -2.],
        [  1.],
        [ -1.],
        ...,
        [-19.],
        [-21.],
        [-14.]],

       [[  2.],
        [  1.],
        [  3.],
        ...,
        [  0.],
        [  2.],
        [  4.]]], dtype=float32)

# A normal (cross-sectional) NN

This model assumes that the data is NOT a time-series data set. It treats the data as cross-sectional and the columns being independent of each other.

In [14]:
model = keras.models.Sequential([
    
    keras.layers.Flatten(input_shape=[36, 1]),
    keras.layers.Dense(36, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
    
])

In [15]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = tf.keras.optimizers.Nadam(learning_rate=0.01)

# If multiclass, use "sparse_categorical_crossentropy" as the loss function
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])


history = model.fit(X_train, y_train, epochs=50,
                    validation_data=(X_test, y_test))

Epoch 1/50
27/27 [==============================] - 1s 11ms/step - loss: 0.9616 - accuracy: 0.6876 - val_loss: 0.7084 - val_accuracy: 0.7154
Epoch 2/50
27/27 [==============================] - 0s 3ms/step - loss: 0.6432 - accuracy: 0.7747 - val_loss: 0.4974 - val_accuracy: 0.8049
Epoch 3/50
27/27 [==============================] - 0s 3ms/step - loss: 0.4642 - accuracy: 0.7782 - val_loss: 0.5270 - val_accuracy: 0.7995
Epoch 4/50
27/27 [==============================] - 0s 3ms/step - loss: 0.4356 - accuracy: 0.7921 - val_loss: 0.5083 - val_accuracy: 0.8022
Epoch 5/50
27/27 [==============================] - 0s 3ms/step - loss: 0.4030 - accuracy: 0.8188 - val_loss: 0.4540 - val_accuracy: 0.8103
Epoch 6/50
27/27 [==============================] - 0s 3ms/step - loss: 0.4052 - accuracy: 0.8072 - val_loss: 0.5229 - val_accuracy: 0.7615
Epoch 7/50
27/27 [==============================] - 0s 3ms/step - loss: 0.4018 - accuracy: 0.8084 - val_loss: 0.4700 - val_accuracy: 0.8076
Epoch 8/50
27/27 [=

In [16]:
# evaluate the model

scores = model.evaluate(X_test, y_test, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.6514373421669006, 0.7804877758026123]

In [17]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


loss: 0.65
accuracy: 78.05%


# Simple RNN with one layer

In [18]:
n_steps = 36
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.SimpleRNN(32, input_shape=[n_steps, n_inputs]),
    keras.layers.Dense(1, activation='sigmoid')
])

In [19]:
from tensorflow.keras.callbacks import EarlyStopping

optimizer = tf.keras.optimizers.Nadam(learning_rate=0.01)

# If multiclass, use "sparse_categorical_crossentropy" as the loss function
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')

history = model.fit(
    X_train, 
    y_train, 
    epochs=50,
    validation_data=(X_test, y_test), 
    callbacks=[early_stop]
)

Epoch 1/50
27/27 [==============================] - 2s 20ms/step - loss: 0.4407 - accuracy: 0.8002 - val_loss: 0.3970 - val_accuracy: 0.8266
Epoch 2/50
27/27 [==============================] - 0s 8ms/step - loss: 0.4173 - accuracy: 0.8002 - val_loss: 0.3955 - val_accuracy: 0.8130
Epoch 3/50
27/27 [==============================] - 0s 8ms/step - loss: 0.4127 - accuracy: 0.8060 - val_loss: 0.4008 - val_accuracy: 0.8103
Epoch 4/50
27/27 [==============================] - 0s 8ms/step - loss: 0.4033 - accuracy: 0.7886 - val_loss: 0.3843 - val_accuracy: 0.8320
Epoch 5/50
27/27 [==============================] - 0s 8ms/step - loss: 0.3901 - accuracy: 0.8188 - val_loss: 0.4112 - val_accuracy: 0.8076
Epoch 6/50
27/27 [==============================] - 0s 8ms/step - loss: 0.3890 - accuracy: 0.8026 - val_loss: 0.4128 - val_accuracy: 0.8022
Epoch 7/50
27/27 [==============================] - 0s 8ms/step - loss: 0.3841 - accuracy: 0.8200 - val_loss: 0.3884 - val_accuracy: 0.8266
Epoch 8/50
27/27 [=

In [20]:
# evaluate the model
scores = model.evaluate(X_test, y_test, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.40958625078201294, 0.7994579672813416]

In [21]:
# extract the accuracy from model.evaluate
print(f"{model.metrics_names[0]:s}: {scores[0]:.2f}")
print(f"{model.metrics_names[1]:s}: {scores[1]*100:.2f}")

loss: 0.41
accuracy: 79.95


In [22]:
# Predictions are probabilities.

predictions = model.predict(X_test)

12/12 [==============================] - 0s 3ms/step


In [23]:
# Rounding the probabilities determines 1 or 0

np.round(predictions)[:10] # show first 10 predictions

array([[0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.]], dtype=float32)

In [24]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, np.round(predictions))

array([[125,  42],
       [ 32, 170]], dtype=int64)

# Deep RNN

**Be careful: when stacking RNN layers, you have to set "return_sequences" to True. This enables the layer to send a "sequence" of values to the next layer -- jut like how it uses a sequence of values for training.**

**Since the last layer is DENSE, it can't take sequence data. Therefore, you CANNOT return sequences from the previous layer. So, remove** `return_sequences` **from previous layer.**

In [25]:
n_steps = 36
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.SimpleRNN(32, return_sequences=True, input_shape=[n_steps, n_inputs] ),
    keras.layers.SimpleRNN(32, return_sequences=True),
    keras.layers.SimpleRNN(32), 
    keras.layers.Dense(1, activation='sigmoid')
])

In [26]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(
    X_train, 
    y_train, 
    epochs=20,
    validation_data = (X_test, y_test), 
    callbacks=[early_stop])  # we defined this early_stop callback function earlier

Epoch 1/20
27/27 [==============================] - 4s 39ms/step - loss: 0.4706 - accuracy: 0.7816 - val_loss: 0.4040 - val_accuracy: 0.8184
Epoch 2/20
27/27 [==============================] - 1s 21ms/step - loss: 0.4325 - accuracy: 0.7863 - val_loss: 0.3998 - val_accuracy: 0.8049
Epoch 3/20
27/27 [==============================] - 1s 20ms/step - loss: 0.4251 - accuracy: 0.7898 - val_loss: 0.4179 - val_accuracy: 0.8157
Epoch 4/20
27/27 [==============================] - 1s 22ms/step - loss: 0.4044 - accuracy: 0.8026 - val_loss: 0.3896 - val_accuracy: 0.8211
Epoch 5/20
27/27 [==============================] - 1s 22ms/step - loss: 0.3907 - accuracy: 0.8235 - val_loss: 0.4118 - val_accuracy: 0.7886
Epoch 6/20
27/27 [==============================] - 1s 32ms/step - loss: 0.3870 - accuracy: 0.8107 - val_loss: 0.4303 - val_accuracy: 0.7859
Epoch 7/20
27/27 [==============================] - 1s 26ms/step - loss: 0.3916 - accuracy: 0.8037 - val_loss: 0.4075 - val_accuracy: 0.8266
Epoch 8/20
27

In [27]:
# evaluate the model

scores = model.evaluate(X_test, y_test, verbose=0)

scores
# In results, first is loss, second is accuracy

[0.433872789144516, 0.8102980852127075]

In [28]:
# extract the accuracy from model.evaluate
print(f"{model.metrics_names[0]:s}: {scores[0]:.2f}")
print(f"{model.metrics_names[1]:s}: {scores[1]*100:.2f}")

loss: 0.43
accuracy: 81.03


# LSTM with one layer

In [29]:
n_steps = 36
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.LSTM(32, input_shape=[n_steps, n_inputs]),
    keras.layers.Dense(1, activation='sigmoid')
])

In [30]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(
    X_train, 
    y_train, 
    epochs=20,
    validation_data=(X_test, y_test),
    callbacks=[early_stop]) # we defined this early_stop callback function earlier

Epoch 1/20
27/27 [==============================] - 3s 36ms/step - loss: 0.4703 - accuracy: 0.7875 - val_loss: 0.3955 - val_accuracy: 0.8238
Epoch 2/20
27/27 [==============================] - 0s 13ms/step - loss: 0.4203 - accuracy: 0.8026 - val_loss: 0.4021 - val_accuracy: 0.8238
Epoch 3/20
27/27 [==============================] - 0s 14ms/step - loss: 0.4141 - accuracy: 0.7979 - val_loss: 0.3986 - val_accuracy: 0.8211
Epoch 4/20
27/27 [==============================] - 0s 14ms/step - loss: 0.4089 - accuracy: 0.8084 - val_loss: 0.3889 - val_accuracy: 0.8211
Epoch 5/20
27/27 [==============================] - 0s 15ms/step - loss: 0.4063 - accuracy: 0.8095 - val_loss: 0.3995 - val_accuracy: 0.8211
Epoch 6/20
27/27 [==============================] - 0s 15ms/step - loss: 0.4013 - accuracy: 0.8072 - val_loss: 0.4113 - val_accuracy: 0.8157
Epoch 7/20
27/27 [==============================] - 0s 14ms/step - loss: 0.3975 - accuracy: 0.8002 - val_loss: 0.4062 - val_accuracy: 0.8293
Epoch 8/20
27

In [31]:
# evaluate the model

scores = model.evaluate(X_test, y_test, verbose=0)
scores

# In results, first is loss, second is accuracy

[0.43111592531204224, 0.794037938117981]

In [32]:
# extract the accuracy from model.evaluate
print(f"{model.metrics_names[0]:s}: {scores[0]:.2f}")
print(f"{model.metrics_names[1]:s}: {scores[1]*100:.2f}")

loss: 0.43
accuracy: 79.40


# LSTM with more layers

In [33]:
n_steps = 36
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.LSTM(32, return_sequences=True, input_shape=[n_steps, n_inputs]),
    keras.layers.LSTM(32, return_sequences=True),
    keras.layers.LSTM(32),
    keras.layers.Dense(1, activation='sigmoid')
])

In [34]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(
    X_train, 
    y_train, 
    epochs=20,
    validation_data=(X_test, y_test), 
    callbacks=[early_stop] # we defined this early_stop callback function earlier
)

Epoch 1/20
27/27 [==============================] - 9s 92ms/step - loss: 0.5189 - accuracy: 0.7375 - val_loss: 0.4127 - val_accuracy: 0.8211
Epoch 2/20
27/27 [==============================] - 1s 41ms/step - loss: 0.4444 - accuracy: 0.7956 - val_loss: 0.4188 - val_accuracy: 0.8103
Epoch 3/20
27/27 [==============================] - 1s 41ms/step - loss: 0.4257 - accuracy: 0.7840 - val_loss: 0.4101 - val_accuracy: 0.8184
Epoch 4/20
27/27 [==============================] - 1s 41ms/step - loss: 0.4193 - accuracy: 0.7886 - val_loss: 0.3975 - val_accuracy: 0.8238
Epoch 5/20
27/27 [==============================] - 1s 42ms/step - loss: 0.4138 - accuracy: 0.8037 - val_loss: 0.4131 - val_accuracy: 0.8076
Epoch 6/20
27/27 [==============================] - 1s 42ms/step - loss: 0.4097 - accuracy: 0.8060 - val_loss: 0.4039 - val_accuracy: 0.8266
Epoch 7/20
27/27 [==============================] - 1s 42ms/step - loss: 0.4085 - accuracy: 0.7933 - val_loss: 0.4082 - val_accuracy: 0.8266
Epoch 8/20
27

In [35]:
# evaluate the model

scores = model.evaluate(X_test, y_test, verbose=0)
scores

# In results, first is loss, second is accuracy

[0.4725901186466217, 0.7859078645706177]

In [36]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


loss: 0.47
accuracy: 78.59%


# GRU with one layer

In [37]:
n_steps = 36
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.GRU(32, input_shape=[n_steps, n_inputs]),
    keras.layers.Dense(1, activation='sigmoid')
])

In [38]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(
    X_train, 
    y_train, 
    epochs=20,
    validation_data=(X_test, y_test),
    callbacks=[early_stop] # we defined this early_stop callback function earlier
)

Epoch 1/20
27/27 [==============================] - 4s 41ms/step - loss: 0.5236 - accuracy: 0.7375 - val_loss: 0.3912 - val_accuracy: 0.8184
Epoch 2/20
27/27 [==============================] - 0s 17ms/step - loss: 0.4159 - accuracy: 0.7909 - val_loss: 0.3947 - val_accuracy: 0.8184
Epoch 3/20
27/27 [==============================] - 0s 18ms/step - loss: 0.4134 - accuracy: 0.7979 - val_loss: 0.3898 - val_accuracy: 0.8238
Epoch 4/20
27/27 [==============================] - 0s 18ms/step - loss: 0.4094 - accuracy: 0.7967 - val_loss: 0.3844 - val_accuracy: 0.8347
Epoch 5/20
27/27 [==============================] - 0s 18ms/step - loss: 0.4072 - accuracy: 0.7967 - val_loss: 0.3944 - val_accuracy: 0.8184
Epoch 6/20
27/27 [==============================] - 0s 18ms/step - loss: 0.4049 - accuracy: 0.8060 - val_loss: 0.4068 - val_accuracy: 0.8266
Epoch 7/20
27/27 [==============================] - 0s 18ms/step - loss: 0.4010 - accuracy: 0.8002 - val_loss: 0.4034 - val_accuracy: 0.8184
Epoch 8/20
27

In [39]:
# evaluate the model

scores = model.evaluate(X_test, y_test, verbose=0)
scores

# In results, first is loss, second is accuracy

[0.4184522032737732, 0.8292682766914368]

In [40]:
# extract the accuracy from model.evaluate
print(f"{model.metrics_names[0]:s}: {scores[0]:.2f}")
print(f"{model.metrics_names[1]:s}: {scores[1]*100:.2f}")

loss: 0.42
accuracy: 82.93


# GRU with more layers

In [41]:
n_steps = 36
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.GRU(32, return_sequences=True, input_shape=[n_steps, n_inputs]),
    keras.layers.GRU(32, return_sequences=True),
    keras.layers.GRU(32),
    keras.layers.Dense(1, activation='sigmoid')
])

In [42]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(
    X_train, 
    y_train, 
    epochs=20,
    validation_data=(X_test, y_test), 
    callbacks=[early_stop] # we defined this early_stop callback function earlier
)

Epoch 1/20
27/27 [==============================] - 9s 92ms/step - loss: 0.4614 - accuracy: 0.7758 - val_loss: 0.3945 - val_accuracy: 0.8157
Epoch 2/20
27/27 [==============================] - 1s 48ms/step - loss: 0.4269 - accuracy: 0.7875 - val_loss: 0.4024 - val_accuracy: 0.8184
Epoch 3/20
27/27 [==============================] - 1s 49ms/step - loss: 0.4221 - accuracy: 0.7979 - val_loss: 0.4068 - val_accuracy: 0.8211
Epoch 4/20
27/27 [==============================] - 1s 48ms/step - loss: 0.4207 - accuracy: 0.7875 - val_loss: 0.3850 - val_accuracy: 0.8238
Epoch 5/20
27/27 [==============================] - 1s 49ms/step - loss: 0.4223 - accuracy: 0.8026 - val_loss: 0.4015 - val_accuracy: 0.8130
Epoch 6/20
27/27 [==============================] - 1s 50ms/step - loss: 0.4151 - accuracy: 0.7921 - val_loss: 0.4103 - val_accuracy: 0.8157
Epoch 7/20
27/27 [==============================] - 1s 50ms/step - loss: 0.4070 - accuracy: 0.7898 - val_loss: 0.4046 - val_accuracy: 0.8266
Epoch 8/20
27

In [43]:
# evaluate the model

scores = model.evaluate(X_test, y_test, verbose=0)
scores

# In results, first is loss, second is accuracy

[0.40711504220962524, 0.8265582919120789]

In [44]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


loss: 0.41
accuracy: 82.66%


# Conv1D

In [45]:
n_steps = 36
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.Conv1D(filters=10, kernel_size=3, strides=2, padding="valid", input_shape=[n_steps, n_inputs]),
    keras.layers.Conv1D(filters=20, kernel_size=3, strides=1, padding="valid", dilation_rate=2),
    keras.layers.LSTM(32, return_sequences=True),
    keras.layers.LSTM(32),
    keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 17, 10)            40        
                                                                 
 conv1d_1 (Conv1D)           (None, 13, 20)            620       
                                                                 
 lstm_4 (LSTM)               (None, 13, 32)            6784      
                                                                 
 lstm_5 (LSTM)               (None, 32)                8320      
                                                                 
 dense_8 (Dense)             (None, 1)                 33        
                                                                 
Total params: 15,797
Trainable params: 15,797
Non-trainable params: 0
_________________________________________________________________


In [46]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(
    X_train, 
    y_train, 
    epochs=20,
    validation_data=(X_test, y_test), 
    callbacks=[early_stop] # we defined this early_stop callback function earlier
)

Epoch 1/20
27/27 [==============================] - 7s 86ms/step - loss: 0.5212 - accuracy: 0.7410 - val_loss: 0.4179 - val_accuracy: 0.8184
Epoch 2/20
27/27 [==============================] - 2s 65ms/step - loss: 0.4519 - accuracy: 0.7933 - val_loss: 0.4261 - val_accuracy: 0.7913
Epoch 3/20
27/27 [==============================] - 2s 64ms/step - loss: 0.4378 - accuracy: 0.7944 - val_loss: 0.4213 - val_accuracy: 0.8076
Epoch 4/20
27/27 [==============================] - 2s 61ms/step - loss: 0.4325 - accuracy: 0.7875 - val_loss: 0.4223 - val_accuracy: 0.8130
Epoch 5/20
27/27 [==============================] - 2s 67ms/step - loss: 0.4301 - accuracy: 0.7979 - val_loss: 0.4426 - val_accuracy: 0.7805
Epoch 6/20
27/27 [==============================] - 2s 61ms/step - loss: 0.4122 - accuracy: 0.8026 - val_loss: 0.4565 - val_accuracy: 0.8049
Epoch 6: early stopping


In [47]:
# evaluate the model

scores = model.evaluate(X_test, y_test, verbose=0)
scores

# In results, first is loss, second is accuracy

[0.45653003454208374, 0.8048780560493469]

In [48]:
# extract the accuracy from model.evaluate
print(f"{model.metrics_names[0]:s}: {scores[0]:.2f}")
print(f"{model.metrics_names[1]:s}: {scores[1]*100:.2f}")

loss: 0.46
accuracy: 80.49
